![Logo](logo.png)
# GKN Aerospace Businesses and Feedback

---

This is a __demonstration__ Jupyter Notebook to showcase how one can use Python/Jupyter to display a live map with GKN's businesses and feedback values around the world. The data is loaded from an excel file that is displayed below.


__Note: __ All Feedback values here are random numbers. They do not, in any sense, reflect the actual values.

In [13]:
import os
import sys
import uuid
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt

sys.path.append(os.path.join(os.getcwd(),"branca-master"))
sys.path.append(os.path.join(os.getcwd(),"folium-master"))
sys.path.append(os.path.join(os.getcwd(),"vincent-master"))

import folium
from folium.plugins import MarkerCluster
from folium import IFrame
import vincent
from ipython_improvements import code_toggler
display(code_toggler(debug=True))

# Load Data from Excel

In [14]:
sample = os.path.join(os.getcwd(),"map list.xlsx")
df = pd.read_excel(sample)
display(df)

,Site,Business,Latitude,Longitude,Competence,Product,GAI Contact,Site Contact,Invoiced Hours YTD,Active/Non Active,Comment,Feedback,Feedback Q-1,Feedback Q-2,Feedback Q-3
0,GKN Aerospace Sweden,AES,58.283500,12.285800,"PLM, ME, Military,Commercial","ICC, IMC, TEC, TRF, RM12",David Orth,Karl-David Pettersson,30198.823217,Active,NaN,7.995085,7.195577,6.476019,5.828417
1,GKN Aerospace Norway,AES,59.668900,9.650200,"ME, CAM/NC",NaN,Kumaran Ganesan,Lium Odd Terje,12647.733963,Active,NaN,7.171506,6.454355,5.808920,5.228028
2,GKN Aerospace Newington,AES,41.697300,-72.722800,"ME, CAM/NC",NaN,Kumaran Ganesan,Oskar Hoguland,27243.785410,Active,NaN,6.394767,5.755290,5.179761,4.661785
3,GKN Aerospace Applied Composites,AES,58.410800,15.621400,NaN,NaN,TBD,NaN,6760.162858,Active,NaN,7.231472,6.508325,5.857492,5.271743
4,GKN Aerospace Cincinnati,AES,39.103100,-84.512000,"ME, CAM/NC",NaN,Kumaran Ganesan,Per Lund,24469.011728,Active,NaN,7.243889,6.519500,5.867550,5.280795
5,GKN Aerospace Cromwell,AES,41.595100,-72.645400,NaN,NaN,David Orth,NaN,66507.158857,Active,NaN,7.299326,6.569394,5.912454,5.321209
6,GKN Aerospace Chem-tronics,ANA,32.794800,-116.962500,NaN,NaN,Kumaran Ganesan,"Correa, Everett",48963.982800,Active,NaN,7.581236,6.823113,6.140801,5.526721
7,GKN Aerospace Manchester,AES,53.480800,-2.242600,CAM,NaN,Kumaran Ganesan,"Dzananovic, Enes",86852.049185,Active,NaN,6.527201,5.874481,5.287033,4.758330
8,GKN Aerospace Mexicali,AES,32.624500,-115.452300,NaN,NaN,TBD,NaN,83718.783328,Active,NaN,8.193577,7.374219,6.636797,5.973117
9,GKN Aerospace Muncie,AES,40.193400,-85.386400,Procurement,NaN,Dharma,xx,21253.938964,Active,NaN,9.544277,8.589850,7.730865,6.957778


In [3]:
def get_random_df():
    """This function will return a random KPI table. This is just for testing purposes."""
    import pandas as pd
    import random
    columns = ["KPI1","KPI2","KPI3"]
    df = pd.DataFrame(columns=columns)
    for i in range(10):
        data = {}
        for column in columns:
            data[column] = random.randint(5,9) + random.random()
        df = df.append(data, ignore_index=True)
    return df.mean()

In [45]:
SF_COORDINATES = (37.76, -122.45)
map = folium.Map(location=SF_COORDINATES, zoom_start=1)
markers = []
popups = []
images_dir = "images"
if not os.path.isdir(images_dir):
    os.makedirs(images_dir)
    
for ix,row in df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    if not pd.isnull(latitude) and not pd.isnull(longitude):
        html = ""
        plottable_columns = []
        for column in row.index:
            value = row[column]
            if not(column  in ["Latitude","Longitude"]) and not(pd.isnull(value)):
                if isinstance(value, str) or (column == "Invoiced Hours YTD"):
                    html = html + "<b>{}:</b> {}<br />".format(column, value)
                else:
                    plottable_columns.append(column)
        if len(plottable_columns)>0:
            for col in plottable_columns:
                if row[col] <= 7:
                    colors.append("#f44336")
                else:
                    colors.append("#4caf50")
            ax = row[plottable_columns].plot(kind="bar", color=colors, grid=True)
            fig = ax.get_figure()
            uuid_ = str(uuid.uuid4())
            image_path = os.path.join(images_dir,"{}.png".format(uuid_))
            ax.set_ylim(0,10)
            ax.set_ylabel("KPI Score out of 10")
            ax.set_yticks(list(range(11)))
            fig.savefig(image_path, dpi=110)
            plt.clf()
            html= html + '<img src="{}" alt="KPI Chart" height="300"><br><hr>'.format(image_path)
        marker = [latitude,longitude]
        markers.append(marker)
        popup = folium.Popup(folium.Html(html, script=True), max_width=1200)
        popups.append(popup)
pt_lyr = folium.FeatureGroup(name = 'pt_lyr')
marker_cluster = MarkerCluster(locations=markers, popups=popups)
pt_lyr.add_child(marker_cluster)
map.add_child(pt_lyr)
display(map)
map.save("gkn_aerospace_businesses.html")

In [ ]:
        site = row["Site"]
        business = row["Business"]
        keyfunctions = row["Key Functions"] if pd.notnull(row["Key Functions"]) else "-"
        gai_contact = row["GAI Contact"] if pd.notnull(row["GAI Contact"]) else "-"
        site_contact = row["Site Contact"] if pd.notnull(row["Site Contact"]) else "-"
        
        feedback = np.round(row["Feedback"],2)
        color = "red" if feedback < 8 else "green"
        kpi_df = get_random_df()
        colors = []
        for col in kpi_df.index:
            if kpi_df[col] <=7:
                colors.append("#f44336")
            else:
                colors.append("#4caf50")
        
        ax = kpi_df.plot(kind="bar", color=colors, grid=True)
        fig = ax.get_figure()
        uuid_ = str(uuid.uuid4())
        image_path = os.path.join(images_dir,"{}.png".format(uuid_))
        
        
        ax.set_ylim(0,10)
        ax.set_ylabel("KPI Score out of 10")
        ax.set_yticks(list(range(11)))
        fig.savefig(image_path, dpi=110)
        plt.clf()
        args = [site, business, keyfunctions, gai_contact, site_contact, color, feedback, image_path]
        html = (
                "<b>Site:</b> {}<br>"
                "<b>Business:</b>: {}<br>"
                "<b>Key Functions:</b>: {}<br>"
                "<b>GAI Contact:</b>: {}<br>"
                "<b>Site Contact:</b>: {}<br>"
                "<b>Feedback:</b>: "
                '<font color="{}">{}</font><br>'
                '<b>KPIs:</b><br><img alt="KPI Graph Shown Here" src="{}" height="300"><br><hr>').format(*args)
        marker = [latitude,longitude]
        markers.append(marker)
        
        popup = folium.Popup(folium.Html(html, script=True), max_width=1200)
        popups.append(popup)
#         popups.append(IFrame(html, width=300,height=180))

pt_lyr = folium.FeatureGroup(name = 'pt_lyr')
marker_cluster = MarkerCluster(locations=markers, popups=popups)
pt_lyr.add_child(marker_cluster)
map.add_child(pt_lyr)
display(map)
map.save("gkn_aerospace_businesses.html")

In [12]:
SF_COORDINATES = (37.76, -122.45)
map = folium.Map(location=SF_COORDINATES, zoom_start=1)
markers = []
popups = []
images_dir = "images"
if not os.path.isdir(images_dir):
    os.makedirs(images_dir)
    
for ix,row in df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    if not pd.isnull(latitude):
        
        site = row["Site"]
        business = row["Business"]
        keyfunctions = row["Key Functions"] if pd.notnull(row["Key Functions"]) else "-"
        gai_contact = row["GAI Contact"] if pd.notnull(row["GAI Contact"]) else "-"
        site_contact = row["Site Contact"] if pd.notnull(row["Site Contact"]) else "-"
        
        feedback = np.round(row["Feedback"],2)
        color = "red" if feedback < 8 else "green"
        kpi_df = get_random_df()
        colors = []
        for col in kpi_df.index:
            if kpi_df[col] <=7:
                colors.append("#f44336")
            else:
                colors.append("#4caf50")
        
        ax = kpi_df.plot(kind="bar", color=colors, grid=True)
        fig = ax.get_figure()
        uuid_ = str(uuid.uuid4())
        image_path = os.path.join(images_dir,"{}.png".format(uuid_))
        
        
        ax.set_ylim(0,10)
        ax.set_ylabel("KPI Score out of 10")
        ax.set_yticks(list(range(11)))
        fig.savefig(image_path, dpi=110)
        plt.clf()
        args = [site, business, keyfunctions, gai_contact, site_contact, color, feedback, image_path]
        html = (
                "<b>Site:</b> {}<br>"
                "<b>Business:</b>: {}<br>"
                "<b>Key Functions:</b>: {}<br>"
                "<b>GAI Contact:</b>: {}<br>"
                "<b>Site Contact:</b>: {}<br>"
                "<b>Feedback:</b>: "
                '<font color="{}">{}</font><br>'
                '<b>KPIs:</b><br><img alt="KPI Graph Shown Here" src="{}" height="300"><br><hr>').format(*args)
        marker = [latitude,longitude]
        markers.append(marker)
        
        popup = folium.Popup(folium.Html(html, script=True), max_width=1200)
        popups.append(popup)
#         popups.append(IFrame(html, width=300,height=180))

pt_lyr = folium.FeatureGroup(name = 'pt_lyr')
marker_cluster = MarkerCluster(locations=markers, popups=popups)
pt_lyr.add_child(marker_cluster)
map.add_child(pt_lyr)
display(map)
map.save("gkn_aerospace_businesses.html")